# 放量涨停

1. n日内有且仅有一个涨停，日线放量6倍以上。
2. 日线放量是过去至少20天内均值的5倍以上。
3. 过去存在一次放量（主力开始收集筹码）
4. 股价为60日内低位 wr < 0.6

<img src="https://images.jieyu.ai/images/202111/20211103225820.png" width="450"/>

In [13]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsi30, rsiday
from alpha.utils import math_round, buy_limit_price
from numpy.lib.stride_tricks import sliding_window_view

await init_notebook()
init_jq()

In [9]:
def scan_buy_limit(n = 20, adv_thres=0.8, secs=None):
    fields = ["open", "close", "high", "low", "volume", "high_limit"]
    
    secs = secs or Securities().choose(['stock'])
    
    rec_limit_per_req = 3000
    secs_per_req = rec_limit_per_req // n
    
    batches = len(secs) // secs_per_req + 1
    
    r1 = []
    for i in range(batches):
        codes = secs[i*secs_per_req:(i+1)*secs_per_req]
        bars = jq.get_price(codes, count=n, end_date=arrow.now().date(), fields=fields, panel=False,skip_paused=True)

        for code in codes:
            bar = bars[bars.code == code]
            
            close = bar['close'].values
            high_limit = bar['high_limit'].values
            volume = bar['volume'].values
            
            reach_buy_limit = np.abs(close - high_limit) < 1e-3
            total = np.count_nonzero(reach_buy_limit)
            last_5_days = np.count_nonzero(reach_buy_limit[-5:])
                        
            # 跳过当天涨停、最近5天涨停超过1次
            if not reach_buy_limit[-1] and total >= 1 and last_5_days <= 1:
                # 最后涨停时的坐标
                pos = np.argwhere(reach_buy_limit).flatten()[-1]
                mn = np.mean(volume[pos-5:pos])
                if mn > 0:
                    lb = volume[pos] / mn
                    r1.append((code, round(lb,1)))
                
    if len(r1) == 0:
        return []
    
    # 去掉半年内大涨的个股
    nbars = 120
    secs_per_req = rec_limit_per_req // nbars
    batches = len(secs) // secs_per_req + 1

    r2 = []
    for i in range(batches):
        recs = r1[i*secs_per_req: (i+1) * secs_per_req]
        codes = [rec[0] for rec in recs]
        
        if len(codes) == 0:
            break
            
        bars = jq.get_price(codes, count=nbars, end_date=arrow.now().date(), panel=False)
        for code, lb in recs:
            bar = bars[bars.code == code]
            close = bar['close'].values
                
            low = np.min(close)
        
            #print(bars.index[pos_high], close[pos_high], low, close[pos_high]/low - 1)
        
            adv = close[-1]/low - 1
            if adv > adv_thres:
                continue

            r2.append((code, Security(code).display_name, adv, lb))
        
    return pd.DataFrame(r2, columns=['code', 'name', 'adv', 'lb'])

In [11]:
class BuyLimitStrategy:
    def __init__(self, n:int):
        self.stats = self.reach_buy_limit_stats(n)
        
    def reach_buy_limit_stats(self, n:int):
        fields = ["open", "close", "high", "low", "volume", "high_limit"]

        secs = Securities().choose(['stock'])

        rec_limit_per_req = 3000
        secs_per_req = rec_limit_per_req // n

        batches = len(secs) // secs_per_req + 1

        r1 = []
        for i in range(batches):
            codes = secs[i*secs_per_req:(i+1)*secs_per_req]
            bars = jq.get_price(codes, count=n, end_date=arrow.now().date(), fields=fields, panel=False,skip_paused=True)

            for code in codes:
                bar = bars[bars.code == code]

                close = bar['close'].values
                high_limit = bar['high_limit'].values
                volume = bar['volume'].values

                reach_buy_limit = np.abs(close - high_limit) < 1e-3
                total = np.count_nonzero(reach_buy_limit)
                if total != 0:
                    r1.append((code, total, reach_buy_limit))

        return r1

    def filter_buy_limit(self, last_m:int, nzt:List, include_last = False):
        results = []
        for code, total, stats in self.stats:
            name = Security(code).display_name
            count = np.count_nonzero(stats[-last_m:])
            if count >= nzt[0] and count <= nzt[1] and not stats[-1]:
                results.append((name, code, total, count))
                
        return pd.DataFrame(results, columns = ["name", "code", "total", "recent"])

In [12]:
s = BuyLimitStrategy(120)
s.filter_buy_limit(5,[1,1])

,name,code,total,recent
0,神州数码,000034.XSHE,2,1
1,皇庭国际,000056.XSHE,8,1
2,常山北明,000158.XSHE,2,1
3,长航凤凰,000520.XSHE,4,1
4,神州信息,000555.XSHE,1,1
...,...,...,...,...
135,东亚药业,605177.XSHG,1,1
136,德才股份,605287.XSHG,2,1
137,晨光新材,605399.XSHG,11,1
138,东鹏饮料,605499.XSHG,2,1


In [89]:
pd.options.display.max_rows=500
df[df.lb > 3].sort_values('lb', ascending=False)

,code,name,adv,lb
8,000419.XSHE,通程控股,0.193069,16.4
234,300939.XSHE,秋田微,0.345277,12.5
391,605060.XSHG,联德股份,0.317246,11.3
50,000955.XSHE,欣龙控股,0.287770,10.3
247,600118.XSHG,中国卫星,0.068715,10.0
381,603856.XSHG,东宏股份,0.466607,8.8
251,600168.XSHG,武汉控股,0.136876,8.7
372,603615.XSHG,茶花股份,0.151282,8.6
233,300933.XSHE,中辰股份,0.554382,8.2
330,601236.XSHG,红塔证券,0.049955,8.0


# 板后三连阳

In [17]:
jq.get_current_tick("000001.XSHE")

Exception: error: 没有stocktick权限
stocktick属于付费模块（仅限机构使用），如果您有购买需求，可添加微信号JQData02申请试用或咨询开通

In [15]:
jq.auth("18694978299", "87Ai2y3LzPOe")

auth success 


In [18]:
tm = "2019-01-01T09:31:00Z"
from datetime import datetime

%timeit datetime.strptime(tm, "%Y-%m-%dT%H:%M:%SZ")

3.95 µs ± 20 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
%timeit arrow.get(tm)

28.4 µs ± 189 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [39]:
import numpy as np
%timeit np.datetime64(tm).astype(datetime)

<magic-timeit>:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


1.09 µs ± 1.69 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [36]:
import ciso8601
%timeit ciso8601.parse_datetime_as_naive(tm)

90.9 ns ± 1.51 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
